In [ ]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

Error: Jupyter notebook failed to launch. 
Error: Jupyter server crashed. Unable to connect. 
Error code from jupyter: 0
Failed to run jupyter as observable with args notebook --no-browser --notebook-dir=C:\Users\comas\AppData\Local\Temp\0f103f53-6285-44a8-9e8b-b2080e258c3b --config=C:\Users\comas\AppData\Local\Temp\0f103f53-6285-44a8-9e8b-b2080e258c3b\jupyter_notebook_config.py --NotebookApp.iopub_data_rate_limit=10000000000.0


In [14]:
fastmode = False
epochs = 500
lr = 0.01
weight_decay = 5e-4
hidden = 16
dropout = 0.5

cuda = torch.cuda.is_available()

np.random.seed(3)
torch.manual_seed(3)
if cuda:
    torch.cuda.manual_seed(3)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data(path="D:/M2/Stage/code/benchmark_embedded_graph_learning/benchmark/gcn/data/cora/", dataset="cora")
print("Finished loading")
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
        "loss= {:.4f}".format(loss_test.item()),
        "accuracy= {:.4f}".format(acc_test.item()))
    return loss_test.item(), acc_test.item()

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
        'loss_train: {:.4f}'.format(loss_train.item()),
        'acc_train: {:.4f}'.format(acc_train.item()),
        'loss_val: {:.4f}'.format(loss_val.item()),
        'acc_val: {:.4f}'.format(acc_val.item()),
        'time: {:.4f}s'.format(time.time() - t))


Loading cora dataset...


In [ ]:
tab_acc = []
tab_loss = []
tab_ep = []

for i in range(1,10):
    # Model and optimizer
    model = GCN(nfeat=features.shape[1],
                nhid=hidden,
                nclass=labels.max().item() + 1,
                dropout=dropout)
    optimizer = optim.Adam(model.parameters(),
                        lr=lr, weight_decay=weight_decay)

    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        labels = labels.cuda()
        idx_train = idx_train.cuda()
        idx_val = idx_val.cuda()
        idx_test = idx_test.cuda()

    # Train model
    t_total = time.time()
    for epoch in range(epochs * i):
        train(epoch)
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

    # Testing
    loss, acc = test()
    tab_acc.append(acc)
    tab_loss.append(loss)
    tab_ep.append(epochs * i)
plt.plot(tab_ep, tab_acc, linewidth=1.0, linestyle="-")
plt.plot(tab_ep, tab_loss, linewidth=1.0, linestyle="-")